In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

In [ ]:
path = ''
db_name = 'github_repos.db'

try:

    conn = sqlite3.connect(path + db_name)

    cur = conn.cursor()
    sql = '''
        CREATE TABLE IF NOT EXISTS repositories (
            repo_name TEXT PRIMARY KEY,
            language TEXT,
            stars INTEGER
        );
    '''
    # SQL文の実行
    cur.execute(sql)
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)
finally:
    # DBへの接続を閉じる
    if 'conn' in locals() and conn:
        conn.close()


def scrape_github_repos(organization_name):
    
    url = f"https://github.com/orgs/google/repositories" 

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"リクエストエラー: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    repo_list = soup.find_all('div', class_='d-flex flex-justify-between border-bottom')
    scraped_data = []
    
    for repo in repo_list:
        name_tag = repo.find('h3').find('a')
        repo_name = name_tag.text.strip() if name_tag else 'N/A'
        
        language_tag = repo.find('span', itemprop='programmingLanguage')
        language = language_tag.text.strip() if language_tag else 'N/A'

        stars_tag = repo.find('a', href=f'/{organization_name}/{repo_name}/stargazers')
        stars_text = stars_tag.find('span').text.strip() if stars_tag and stars_tag.find('span') else '0'
        
        def convert_stars_to_int(star_str):
            star_str = star_str.replace(',', '')
            if 'k' in star_str.lower():
                return int(float(star_str.lower().replace('k', '')) * 1000)
            else:
                try:
                    return int(star_str)
                except ValueError:
                    return 0

        stars = convert_stars_to_int(stars_text)
            
        data = (repo_name, language, stars)
        scraped_data.append(data)
        
        print(f"   -> 取得: {repo_name} | スター: {stars}")
        
        time.sleep(1) 
        
    return scraped_data

repo_data = scrape_github_repos('google')

try:
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    cur.execute('DELETE FROM repositories')

    sql = "INSERT INTO repositories (repo_name, language, stars) VALUES (?, ?, ?);"

    cur.executemany(sql, repo_data)
    
    conn.commit()
    print(f'{len(repo_data)}件のデータがDBに挿入')

except sqlite3.Error as e:
    print('エラーが発生しました:', e)
finally:
    if 'conn' in locals() and conn:
        conn.close()


#保存したデータのSELECT表示
print("\n## 3. 保存データのSELECT表示処理")
try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)
    cur = conn.cursor()

    # データを参照するSQL: スター数が多い順に並び替える
    sql = "SELECT repo_name, language, stars FROM repositories ORDER BY stars DESC;"

    # SQL文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

else:
    results = cur.fetchall()
    if results:
        print(f"{'リポジトリ名':<40} | {'主要な言語':<20} | {'スターの数':>10}")
        print("-" * 75)
        for row in results:
            name, lang, stars = row
            print(f"{name:<40} | {lang:<20} | {stars:>10,}")
        print('SELECT文の実行とデータ表示が完了しました。')
    else:
        print('データがデータベースに見つかりませんでした。')

finally:
    if 'conn' in locals() and conn:
        conn.close()

0件のデータがDBに挿入

## 3. 保存データのSELECT表示処理
データがデータベースに見つかりませんでした。
